In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.datasets import load_iris
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

### Check if host machine has GPU available

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


### Load Iris data

In [3]:
data = load_iris()
df = pd.DataFrame(data['data'], columns=data.feature_names)
df['species'] = data.target
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### Custom DataSet class
Return features as array with shape (`input_features`,)

In [4]:
class TabularDataset(Dataset):
    """Tabular dataset from X and y numpy arrays"""

    def __init__(self, X, y):
        """
        Args:
            X (array): Features (n, p)
            y (array): Response (n,)
        """
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        features = torch.tensor(self.X[idx,:]).float()
        response = torch.tensor(self.y[idx]).long()
        
        return features, response

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1].values, df.species.values,
                                                    test_size=.1,
                                                    random_state=42)

In [6]:
train = TabularDataset(X_train, y_train)
test = TabularDataset(X_test, y_test)

trainloader = DataLoader(train, batch_size=5)
testloader = DataLoader(test, batch_size=len(test))

In [7]:
# sort of equivalent to pandas' value_counts()
Counter(test.y)

Counter({1: 6, 0: 6, 2: 3})

In [8]:
class Net(nn.Module):
    # define nn
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.fc3(x)
        x = F.log_softmax(x, dim=-1)

        return x

In [9]:
net = Net()
net.to(device)

criterion = nn.CrossEntropyLoss()  # cross entropy loss
optimizer = torch.optim.SGD(net.parameters(), lr=.01, momentum=.9)

for epoch in range(10):  # loop over the dataset multiple times
    
    # turns on drouput
    net.train()

    running_loss = 0.0
    for data in tqdm(trainloader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # turn off gradients for validation, saves memory and computations
    with torch.no_grad():
        
        # turn off dropout
        net.eval()
        inputs, labels = iter(testloader).next()
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs).to('cpu')
        loss = criterion(outputs, labels)
        _, y_pred = torch.max(outputs.data, 1)
        acc = (y_pred.numpy()==labels.to('cpu').numpy()).mean()
            
        print(f'Epoch #{epoch+1} | test loss: {loss:.3f} | test acc: {acc:.3f}')
    

print('\nFinished Training')


Epoch #1 | test loss: 0.302 | test acc: 0.933



Epoch #2 | test loss: 0.216 | test acc: 0.933



Epoch #3 | test loss: 0.423 | test acc: 0.733



Epoch #4 | test loss: 0.562 | test acc: 0.733



Epoch #5 | test loss: 0.146 | test acc: 0.933



Epoch #6 | test loss: 0.203 | test acc: 0.867



Epoch #7 | test loss: 0.436 | test acc: 0.733



Epoch #8 | test loss: 0.172 | test acc: 0.933



Epoch #9 | test loss: 0.105 | test acc: 1.000



Epoch #10 | test loss: 0.096 | test acc: 0.933

Finished Training
